# T6. WALL RECONSTRUCTION

In this script, we reconstruct parametric wall geometries from the instance segmentation and reference heights.
Specifically, we need:
 - T2: instances of walls, ceilings and other objects
 - T5: reference levels

## LIBRARIES

In [2]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context 
import utils as utl
import utils.t6_utils as t6


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

## INPUTS

In [5]:
#paths
path=Path(os.getcwd()).parents[2] # with MB this is 2

input_folder_t4=path/'data'/'t4'/'test' 
# input_folder_t4=path/'data'/'t4'/'train' 

input_folder_t5=path/'data'/'t5'/'test' 
# input_folder_t5=path/'data'/'t5'/'train' 


class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t6'/'test'
# output_folder=path/'data'/'t6'/'train'

os.makedirs(output_folder, exist_ok=True)

#thresholds
t_level=0.5 #max distance to reference levels to adopt their height
t_inliers=0.3 # % inliers for a wall to consider its own members or the neirby ceilings and floors for its position
t_distance=0.7 #max distance to find opposite wallfaces
t_thickness=0.127 #max thickness of a wall
t_trim=0.3 #max distance to trim a wall
t_extend=0.7 #max distance to extend a wall

Import Classes

In [6]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


## PROCESSING

In [36]:
point_cloud_files=utl.get_list_of_files(input_folder_t4,'.laz')
level_files=utl.get_list_of_files(input_folder_t5,'.ttl')


for f_pcd,f_rdf in zip(point_cloud_files[0:1],level_files[0:1]): #only read the first one
    
    print(f'processing {ut.get_filename(f_pcd)}...') 
    las = laspy.read(f_pcd) #if 'las' not in globals() else las
    pcd=gmu.las_to_pcd(las) #if 'pcd' not in globals() else pcd # this is the slowest step
        
    #read point cloud nodes
    pcdNodes=utl.match_graph_with_las(f_pcd,las,pcd,class_dict,getResources=True,getNormals=True)
    
    #read levelNodes
    levelNodes=tl.graph_path_to_nodes(f_rdf)
    for n in levelNodes:
        n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(n.orientedBounds)))

    #get wallNodes
    wallNodes=[n for n in pcdNodes if n.class_id==2]
    
    #get ceiling and floor nodes
    referenceNodes=[n for n in pcdNodes if n.class_id in[0,1]]
    print(f'{len(wallNodes)} pcdNodes and {len(levelNodes)} levels found')  
    
    #estimate parameters
    print('Estimating base_constraint...')
    t6.compute_base_constraint(wallNodes,levelNodes,threshold_level_height=t_level) #there is a weakness here that takes the full point cloud while this is affected by outliers
    print('Estimating top_constraint...')
    t6.compute_top_constraint(wallNodes,levelNodes,threshold_level_height=t_level)
    print('Estimating wall_orientation...')
    t6.compute_wall_orientation(wallNodes,referenceNodes,t_thickness=t_thickness,t_distance=t_distance,t_inliers=t_inliers) 
    print('Estimating wall_thickness...') 
    t6.compute_wall_thickness(wallNodes,t_thickness=t_thickness,t_distance=t_distance)
    print('Estimating wall_axis...')
    t6.compute_wall_axis(wallNodes)
    print('Estimating wall_topology...') 
    t6.trim_and_extend_wall_nodes(wallNodes,t_trim=t_trim,t_extend=t_extend) 
    print('Estimating wall_geometry...')
    t6.compute_wall_geometry(wallNodes)
    
    #write this information to the 3D detection json
    json_data=t6.walls_to_json(wallNodes,f_pcd)
    with open(os.path.join(output_folder,f'{ut.get_filename(f_pcd)}_walls.json'), 'w') as file:
        json.dump(json_data, file, indent=4)
    print("JSON data written to file:", os.path.join(output_folder,f'{ut.get_filename(f_pcd)}_walls.json') )
    
    
    #write the walls to obj
    utl.write_obj_with_submeshes(os.path.join(output_folder,f'{ut.get_filename(f_pcd)}_walls.obj') , [n.wall for n in wallNodes], [n.name for n in wallNodes])
    
    #create BIMNodes
    wallNodesBIM=[]
    for i,n in enumerate(wallNodes):
        b=BIMNode(subject=n.subject,
                  name=n.name,
                  derivedFrom=pcdNodes[i].subject,
                    resource=n.wall,
                    object_id=n.object_id,
                    class_id=n.class_id,    
                    class_name=n.class_name,
                    base_constraint=n.base_constraint.subject,
                    base_constraint_name=n.base_constraint.name,
                    base_offset=np.round(n.base_offset,3),
                    top_constraint=n.top_constraint.subject,
                    top_constraint_name=n.top_constraint.name,
                    top_offset=np.round(n.top_offset,3),
                    height=np.round(n.height,3), #gt
                    width=np.round(n.wallThickness,3), #gt
                    wallLength=np.round(n.wallLength,3), #gt
                    normal=np.round(n.normal,3),
                    axis=n.axis,
                    start_pt=np.round(np.asarray(n.axis.points)[0],3), #gt
                    end_pt=np.round(np.asarray(n.axis.points)[1],3), #gt
                    neighbor_wall_ids_at_start=n.neighbor_wall_ids_at_start,
                    neighbor_wall_ids_at_end=n.neighbor_wall_ids_at_end,
                    color=n.color)
        wallNodesBIM.append(b)        
    graphPath=    os.path.join(output_folder,f'{ut.get_filename(f_pcd)}_walls.ttl')
    new_graph=tl.nodes_to_graph(wallNodesBIM,graphPath=graphPath,save=True)

processing 08_ShortOffice_01_F1_small_pred...
Function match_graph_with_las took 16.7339 seconds to execute.
111 pcdNodes and 5 levels found
Estimating base_constraint...
name: 08_ShortOffice_01_F1_small_pred_walls_201, base_constraint: level_10, base_offset: -0.012137923546972337
name: 08_ShortOffice_01_F1_small_pred_walls_202, base_constraint: level_10, base_offset: -0.5121379235469723
name: 08_ShortOffice_01_F1_small_pred_walls_204, base_constraint: level_10, base_offset: -0.012137923546972337
name: 08_ShortOffice_01_F1_small_pred_walls_205, base_constraint: level_10, base_offset: 0.007862076453027663
name: 08_ShortOffice_01_F1_small_pred_walls_206, base_constraint: level_10, base_offset: 0.31008207645302766
name: 08_ShortOffice_01_F1_small_pred_walls_212, base_constraint: level_10, base_offset: -0.012137923546972337
name: 08_ShortOffice_01_F1_small_pred_walls_213, base_constraint: level_10, base_offset: 0.007862076453027663
name: 08_ShortOffice_01_F1_small_pred_walls_214, base_cons

In [8]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallNodes if n.resource is not None])
# o3d.visualization.draw_geometries([joined_pcd]+[n.resource for n in levelNodes])

In [9]:
# o3d.visualization.draw_geometries([joined_pcd]+
#                                   [n.orientedBoundingBox for n in wallNodes]+
#                                   [n.axis for n in wallNodes]+
#                                   [o3d.geometry.PointCloud(o3d.utility.Vector3dVector(n.boundaryPoints)) for n in wallNodes])

In [39]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallNodes if n.resource is not None])
# sampled_ceilings_and_floors=gmu.sample_geometry(gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in pcdNodes if n.class_id in[0,1]]))[0]
# for n in wallNodes:
#     if n.flipped:
#         n.wallBox.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([joined_pcd]+
                                #   [sampled_ceilings_and_floors]+
                                  [n.wallBox for n in wallNodes]+
                                  [n.axis for n in wallNodes])

spawn potential connections (optional)

In [181]:
# weight_intersection=4
# weight_orthogonal=2
# weight_direct=1

# t_intersection_extension=2
# t_direct_extension=0.5
# t_ortho_extension=0.4

# potential_connections=t6.compute_potential_wall_connections(wallNodesBIM,
#                                       t_intersection_extension=t_intersection_extension,
#                                       t_direct_extension=t_direct_extension,
#                                       t_ortho_extension=t_ortho_extension,
#                                       weight_intersection=weight_intersection,
#                                       weight_orthogonal=weight_orthogonal,
#                                       weight_direct=weight_direct)

# print(f'potential connections: {np.sum([len(n.potential_connections) for n in wallNodesBIM]   )} ')


potential connections: 123 


In [183]:
# intersections = gmu.join_geometries([l._resource.paint_uniform_color([1,0,0]) for n in wallNodesBIM for l in n.potential_connections if len(n.potential_connections) > 0 and l.weight == weight_intersection])
# ortho= gmu.join_geometries([l._resource.paint_uniform_color([0,1,0]) for n in wallNodesBIM for l in n.potential_connections if len(n.potential_connections) > 0 and l.weight == weight_orthogonal])
# direct= gmu.join_geometries([l._resource.paint_uniform_color([1,1,0]) for n in wallNodesBIM for l in n.potential_connections if len(n.potential_connections) > 0 and l.weight == weight_direct])
# joined_pcd_base=gmu.join_geometries([n.new_axis.paint_uniform_color([0.3,0.3,0.3]) for n in wallNodesBIM if n.resource is not None])


# o3d.visualization.draw_geometries([joined_pcd_base]+[intersections,ortho,direct])#[intersection1,intersection2]+

In [180]:
# # wallNodesBIM[0].axis.paint_uniform_color([1,0,0])
# intersection_pcd=o3d.geometry.PointCloud()
# intersection_pcd.points=o3d.utility.Vector3dVector([item for sublist in [n.intersection_points for n in wallNodesBIM if getattr(n,'intersection_points',None) is not None] for item in sublist])
# intersection_pcd.paint_uniform_color([1,0,0])
# joined_lines1=gmu.join_geometries([n.axis for n in wallNodesBIM if n.resource is not None])
# joined_lines2=gmu.join_geometries([n.new_axis for n in wallNodesBIM if n.resource is not None])


# o3d.visualization.draw_geometries([intersection_pcd,joined_lines2])#[intersection1,intersection2]+

## PROCESS TRAINING DATA

In [87]:
#match the graphs with the training data

input_folder_gt=path/'data'/'t1'/'train'

gt_files=utl.get_list_of_files(input_folder_gt,'.ttl')
gt_files_obj=utl.get_list_of_files(input_folder_gt,'.obj')
for f_gt,f_gt_obj in zip(gt_files[6:7],gt_files_obj[6:7]): #only read the first one
    
    #import objects
    mesh_dict=utl.load_obj_and_create_meshes(f_gt_obj)
    
    #import graph
    pcdNodes_gt=tl.graph_path_to_nodes(graphPath=str(f_gt))
    pcdNodes_gt_walls=[n for n in pcdNodes_gt if n.class_id==2]

    #add objects to the nodes
    for n in pcdNodes_gt_walls:
        mesh=next((mesh for name, mesh in mesh_dict.items() if name == n.name),None)
        n.resource=mesh
        n.lineset=o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
        n.lineset.paint_uniform_color(ut.literal_to_array(n.color))
        #compute the normal from start to end point
        n.resource.compute_triangle_normals()
        n.normal =np.asarray( n.resource.triangle_normals)[11]
    print(f'{len(pcdNodes_gt_walls)} pcdNodes_gt_walls detected!') # there are only 144 walls in the training data, yet 161 are found here
    
    #match the walls with the ground truth
    for n in wallNodesBIM:
        
        #create lineset
        n.lineset=o3d.geometry.LineSet.create_from_triangle_mesh(n.resource)
        n.lineset.paint_uniform_color(ut.literal_to_array(n.color))
                
        #find the corresponding ground truth wall 
        pose=n.cartesianTransform[:3,3]
        distances=[np.linalg.norm(pose-n.cartesianTransform[:3,3]) for n in pcdNodes_gt_walls]
        idx=np.argmin(distances)
        n_gt=pcdNodes_gt_walls[idx]
        n_gt.color=n.color
        n.corresponding_gt=n_gt.name
        n.corresponding_normal=n_gt.normal
        n.corresponding_neighbor_wall_ids_at_start_gt=ut.literal_to_list(n_gt.neighbor_wall_ids_at_start)
        n.corresponding_neighbor_wall_ids_at_start_gt=[int(x) for x in n.corresponding_neighbor_wall_ids_at_start_gt if x!=None] if n.corresponding_neighbor_wall_ids_at_start_gt is not None else [0]
        n.corresponding_neighbor_wall_ids_at_end_gt=ut.literal_to_list(n_gt.neighbor_wall_ids_at_end)
        n.corresponding_neighbor_wall_ids_at_end_gt=[int(x) for x in n.corresponding_neighbor_wall_ids_at_end_gt if x!=None] if n.corresponding_neighbor_wall_ids_at_end_gt is not None else [0]

        #create a line from the n.resource.get_center() along the n.normal in red and along the n_gt.normal in green
        # line=o3d.geometry.LineSet()
        # line.points=o3d.utility.Vector3dVector([n.resource.get_center(),n.resource.get_center()+n.normal])
        # line.lines=o3d.utility.Vector2iVector([[0,1]])
        # line.colors=o3d.utility.Vector3dVector([[1,0,0]])
        # n.lineset+=line
        # line=o3d.geometry.LineSet()
        # line.points=o3d.utility.Vector3dVector([n.resource.get_center(),n.resource.get_center()+n_gt.normal])
        # line.lines=o3d.utility.Vector2iVector([[0,1]])
        # line.colors=o3d.utility.Vector3dVector([[0,1,0]])
        # n.lineset+=line
        
        
        #make comparison
        n.startpoint_diff=np.round(np.min(np.array([np.linalg.norm(ut.literal_to_array(n_gt.start_pt)[:2]-n.start_pt[:2]),np.linalg.norm(ut.literal_to_array(n_gt.end_pt)[:2]-n.start_pt[:2])])),2)
        n.endpoint_diff=np.round(np.min(np.array([np.linalg.norm(ut.literal_to_array(n_gt.end_pt)[:2]-n.end_pt[:2]),np.linalg.norm(ut.literal_to_array(n_gt.start_pt)[:2]-n.end_pt[:2])])),2)
        n.height_diff=np.round(n_gt.height-n.height,2) #CVPR height is arbitrary and not based on geometries
        n.width_diff=np.round(n_gt.width-n.width,2)
        n.normal_diff=np.round(1-np.abs(np.dot(n_gt.normal,n.normal)),2)
        print(f'wall {n.get_name()} with length {n.wallLength}, with a startpoint_diff of {n.startpoint_diff}, endpoint_diff of {n.endpoint_diff}, height_diff of {n.height_diff}, wallThickness_diff of {n.width_diff} (gt width is {np.round(n_gt.width,2)}), normal_diff of {n.normal_diff}')

    for n in wallNodesBIM:
        
        #find corresponding neighbors
        n.corresponding_neighbor_wall_ids_at_start=[]
        n.corresponding_neighbor_wall_ids_at_end=[]
        for id in n.neighbor_wall_ids_at_start:
            neighbor=next((m for m in wallNodesBIM if m.object_id==id),None)
            node=next((m for m in pcdNodes_gt_walls if m.name==neighbor.corresponding_gt),None)
            n.corresponding_neighbor_wall_ids_at_start.append(node.object_id)
        for id in n.neighbor_wall_ids_at_end:
            neighbor=next((m for m in wallNodesBIM if m.object_id==id),None)
            node=next((m for m in pcdNodes_gt_walls if m.name==neighbor.corresponding_gt),None)
            n.corresponding_neighbor_wall_ids_at_end.append(node.object_id)
        #compute overlap between n.corresponding_neighbor_wall_ids_at_start_gt and n.corresponding_neighbor_wall_ids_at_start
        n.overlap_start=len(set(n.corresponding_neighbor_wall_ids_at_start_gt).intersection(n.corresponding_neighbor_wall_ids_at_start))/len(n.corresponding_neighbor_wall_ids_at_start_gt)
        n.overlap_end=len(set(n.corresponding_neighbor_wall_ids_at_end_gt).intersection(n.corresponding_neighbor_wall_ids_at_end))/len(n.corresponding_neighbor_wall_ids_at_end_gt)
        # print(f'wall {n.get_name()} with overlap_start of {n.corresponding_neighbor_wall_ids_at_start} and gt {n.corresponding_neighbor_wall_ids_at_start_gt}')
        
    #general comparison    
    print(f'len of elements width_def <0.03: {len([e for e in wallNodesBIM if np.abs(e.width_diff)<=0.03])}/{len(wallNodesBIM)}')
    print(f'len of elements height_diff<0.1: {len([e for e in wallNodesBIM if np.abs(e.height_diff)<=0.1])}/{len(wallNodesBIM)}')
    print(f'len of elements normal_diff<0.05: {len([e for e in wallNodesBIM if np.abs(e.normal_diff)<=0.05])}/{len(wallNodesBIM)}')    
    print(f'len of elements axis location<0.1: {len([e for e in wallNodesBIM if (np.average(np.abs([e.startpoint_diff,e.endpoint_diff]))<=0.1)])}/{len(wallNodesBIM)}')  
    print(f'len of computed elements with t_thickness: {len([e for e in wallNodesBIM if e.width ==t_thickness])}/{len(wallNodesBIM)} vs gt elements {len([e for e in pcdNodes_gt_walls if e.width == t_thickness])}/{len(pcdNodes_gt_walls)}')
    print(f'len of elements with overlap_start>0.5: {len([e for e in wallNodesBIM if e.overlap_start>=0.5])}/{len(wallNodesBIM)}')
    print(f'len of elements with overlap_start>0.5: {len([e for e in wallNodesBIM if e.overlap_end>=0.5])}/{len(wallNodesBIM)}')
    
    
    #general comparison  
    print('in percentages')  
    print(f'len of elements width_def <0.03: {np.round(100*len([e for e in wallNodesBIM if np.abs(e.width_diff)<=0.03])/len(wallNodesBIM),1)}%')
    print(f'len of elements height_diff<0.1: {np.round(100*len([e for e in wallNodesBIM if np.abs(e.height_diff)<=0.1])/len(wallNodesBIM),1)}%')
    print(f'len of elements normal_diff<0.05: {np.round(100*len([e for e in wallNodesBIM if np.abs(e.normal_diff)<=0.05])/len(wallNodesBIM),1)}%')    
    print(f'len of elements axis location<0.1: {np.round(100*len([e for e in wallNodesBIM if (np.average(np.abs([e.startpoint_diff,e.endpoint_diff]))<=0.1)])/len(wallNodesBIM),1)}%')  
    print(f'len of elements with overlap_start>0.5: {np.round(100*np.average((len([e for e in wallNodesBIM if e.overlap_start>=0.5]),len([e for e in wallNodesBIM if e.overlap_end>=0.5])))/len(wallNodesBIM),1)}%')
    

204 pcdNodes_gt_walls detected!
wall 35_Lab_02_F2_small1_walls_2000 with length 61.595, with a startpoint_diff of 34.63, endpoint_diff of 27.82, height_diff of 0.47, wallThickness_diff of -6.82 (gt width is 0.19), normal_diff of 0.9
len of elements width_def <0.03: 0/1
len of elements height_diff<0.1: 0/1
len of elements normal_diff<0.05: 0/1
len of elements axis location<0.1: 0/1
len of computed elements with t_thickness: 0/1 vs gt elements 47/204
len of elements with overlap_start>0.5: 0/1
len of elements with overlap_start>0.5: 0/1
in percentages
len of elements width_def <0.03: 0.0%
len of elements height_diff<0.1: 0.0%
len of elements normal_diff<0.05: 0.0%
len of elements axis location<0.1: 0.0%
len of elements with overlap_start>0.5: 0.0%


In [52]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallNodes if n.resource is not None])

# # joined_pcd=gmu.join_geometries([n.lineset.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallNodesBIM if n.resource is not None])
# joined_pcd_bim=gmu.join_geometries([n.lineset.paint_uniform_color([1,0,0]) for n in wallNodesBIM if np.abs(n.width_diff) >0.03])

# # joined_pcd_gt=gmu.join_geometries([n.lineset.paint_uniform_color(ut.literal_to_array(n.color)) for n in pcdNodes_gt_walls if n.resource is not None])
# joined_pcd_gt=gmu.join_geometries([n.lineset.paint_uniform_color([0.3,0.3,0.3]) for n in pcdNodes_gt_walls if n.resource is not None])

# o3d.visualization.draw_geometries([joined_pcd_bim,joined_pcd]+[joined_pcd_gt])

In [ ]:
# o3d.visualization.draw_geometries([joined_pcd]+
#                                   [n.wallBox for n in wallNodes]+
#                                   [n.axis for n in wallNodes]+
#                                   [o3d.geometry.PointCloud(o3d.utility.Vector3dVector(n.boundaryPoints)) for n in wallNodes])